In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import datetime
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript, clear_output

# Configurar Pandas para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [2]:
def clasificacion_a_edad(clasificacion):
    if clasificacion in ('G', 'PG', 'ATP', 'ATPR', 'A', 'AA'):
        return 0
    elif clasificacion == 'B':
        return 12
    elif clasificacion in ('PG-13', 'P-13', 'P-13R'):
        return 13
    elif clasificacion == 'B-15':
        return 15
    elif clasificacion in ('P-16', 'P16R'):
        return 16
    elif clasificacion == 'R':
        return 17
    elif clasificacion in('NC-17', 'C', 'P-18'):
        return 18
    else:
        return 21  # Para clasificaciones desconocidas o no estándar, se asume una edad adulta

In [3]:
# Filtrar películas basadas en la edad del usuario y la clasificación
def filtrar_peliculas_por_edad(peliculas, edad_usuario):
    return peliculas[peliculas['Clasificacion'].apply(clasificacion_a_edad) <= edad_usuario]

In [4]:
def predict_movies2(user_profile, movies_data):
    # Preparar los inputs para el modelo
    inputs = [np.concatenate([user_profile, movies_data.iloc[i, 4:-11].values, [movies_data.iloc[i, -11]]]) for i in range(len(movies_data))]
    
    # Imprimir los primeros inputs para visualización
    #print("Ejemplo de inputs enviados al modelo:")
    #for input_example in inputs[:10]:  # Cambia el 5 a otro número si deseas ver más ejemplos
    #    print(input_example)
    
    # Hacer las predicciones
    predictions = recomendadorknn.predict_proba(inputs)
    movies_data['probabilidad_gusto'] = [p[1] for p in predictions]
    
    # Ordenar las películas por la probabilidad de que al usuario le gusten
    sorted_movies = movies_data.sort_values(by='probabilidad_gusto', ascending=False)
    return sorted_movies

In [5]:
def ObtenerRecomendacion():
    with output_vista:
        output_vista.clear_output()
        # Buscar películas en recomendacioncartelera que no están en peliculasrecomendadas
        recomendadas = peliculasrecomendadas['Titulo'].unique()
        pelicula_nueva = recomendacioncartelera[~recomendacioncartelera['Titulo'].isin(recomendadas)].iloc[0]
    
        # Extraer detalles de la película recomendada
        titulo = pelicula_nueva['Titulo']
        o_titulo = pelicula_nueva['O_Titulo']
        clasificacion = pelicula_nueva['Clasificacion']
        duracion = pelicula_nueva['Duracion']
        calificacion = round(pelicula_nueva['Calificacion'] * 100, 1)
    
        # Extraer géneros de la película
        generos = ['Acción', 'Aventura', 'Animación', 'Comedia', 'Crimen', 'Documental',
                   'Drama', 'Familiar', 'Fantasía', 'Historia', 'Horror', 'Musical',
                   'Misterio', 'Romance', 'Ciencia Ficción', 'Suspense', 'Guerra', 'Western']
        generos_pelicula = ', '.join([g for g, p in zip(generos, pelicula_nueva[4:22]) if p == 1])
    
        # Extraer los cines donde se exhibe la película y formatear la salida
        cines = ['Gran Patio Zaragoza', 'Juárez Centro Mall', 'La Monumental', 'Las Américas Juárez', 
                 'Las Misiones', 'Pinocelli', 'Plaza Areno', 'Plaza Sendero Las Torres', 
                 'Sendero Ciudad Juárez', 'VIP Las Misiones']
        cines_exhibicion = '\n'.join([f"Cinépolis {c}" for c in cines if pelicula_nueva[c] == 1])
    
        # Mostrar recomendación
        print("\nTe recomendamos ver:\n")
        print(f"{titulo} ({o_titulo})\nGéneros: {generos_pelicula}\nClasificación: {clasificacion}\nCalificación: {calificacion}%")
        if duracion > 0:
            print(f"Duración: {duracion} minutos.\n")
        else:
            print("Duración: Desconocida.\n")
        print("Puedes ver esta película en:\n" + cines_exhibicion + "\n")
    
        # Agregar a peliculasrecomendadas
        id_max = peliculasrecomendadas['ID'].max() + 1 if not peliculasrecomendadas.empty else 1
        pelicula_nueva['ID'] = id_max
        # Asegúrate de copiar solo las columnas necesarias para peliculasrecomendadas
        peliculasrecomendadas.loc[peliculasrecomendadas.shape[0]] = pelicula_nueva[list(peliculasrecomendadas.columns)]

In [6]:
def CalificarPelicula():
    global peliculasrecomendadas, recomendacioncartelera, dfregistro, arreglo_perfil

    if peliculasrecomendadas.empty:
        with output_vista:
            clear_output()
            print("No hay películas recomendadas para calificar en este momento.")
            return

    with output_vista:
        clear_output()
        print("Cuál de las siguientes películas que te recomendamos viste?\n")
        
        for index, row in peliculasrecomendadas.iterrows():
            generos_pelicula = ', '.join([generos[i] for i, pres in enumerate(row.iloc[5:23]) if pres == 1])
            duracion_pelicula = "Desconocida" if row['Duracion'] == 0 else f"{row['Duracion']} minutos"
            pelicula_info = f"<b>{row['Titulo']} ({row['O_Titulo']})</b><br>Géneros: {generos_pelicula}<br>Clasificación: {row['Clasificacion']}<br>Duración: {duracion_pelicula}."
            pelicula_label = widgets.HTML(value=pelicula_info)
            btn_pelicula = widgets.Button(description="Calificar esta película")
            
            btn_pelicula.on_click(lambda btn, idx=index: calificar_interfaz(idx))
            display(widgets.VBox([pelicula_label, btn_pelicula]))

In [7]:
def calificar_interfaz(seleccion):
    pelicula_seleccionada = peliculasrecomendadas.iloc[seleccion]
    
    with output_vista:
        clear_output()
        generos_pelicula = ', '.join([generos[i] for i, pres in enumerate(pelicula_seleccionada.iloc[5:23]) if pres == 1])
        duracion_pelicula = "Desconocida" if pelicula_seleccionada['Duracion'] == 0 else f"{pelicula_seleccionada['Duracion']} minutos"
        pelicula_info = f"¿Qué te pareció la película <b>{pelicula_seleccionada['Titulo']} ({pelicula_seleccionada['O_Titulo']})</b><br>Géneros: {generos_pelicula}<br>Clasificación: {pelicula_seleccionada['Clasificacion']}<br>Duración: {duracion_pelicula}."
        display(widgets.HTML(value=pelicula_info))
        
        opciones_respuesta = ['¡La odié!', 'No me gustó', 'Sí me gustó', '¡La amé!']
        respuesta = widgets.RadioButtons(
            options=opciones_respuesta,
            description='Respuesta:',
            disabled=False
        )
        
        btn_enviar_respuesta = widgets.Button(description="Enviar respuesta")
        btn_enviar_respuesta.on_click(lambda btn: enviar_respuesta(respuesta.value, seleccion, pelicula_seleccionada))
        
        display(respuesta, btn_enviar_respuesta)

In [8]:
def enviar_respuesta(respuesta, seleccion, pelicula_seleccionada):
    ajustes = {'¡La odié!': -0.1, 'No me gustó': -0.05, 'Sí me gustó': 0.05, '¡La amé!': 0.1}
    ajuste = ajustes[respuesta]
    actualizar_perfil(seleccion, ajuste, respuesta, pelicula_seleccionada)
    eliminar_pelicula(seleccion)

In [9]:
def actualizar_perfil(seleccion, ajuste, respuesta, pelicula_seleccionada):
    global arreglo_perfil, dfregistro, dfusuarios

    for i in range(18):
        if pelicula_seleccionada.iloc[i + 5] == 1:
            arreglo_perfil[i] = max(0, min(1, arreglo_perfil[i] + ajuste))
    
    nuevo_registro = {f'u_{generos[i].replace(" ", "_").lower()}': arreglo_perfil[i] for i in range(18)}
    nuevo_registro.update({f'p_{generos[i].replace(" ", "_").lower()}': pelicula_seleccionada.iloc[i + 5] for i in range(18)})
    nuevo_registro['Calificacion'] = pelicula_seleccionada['Calificacion']
    nuevo_registro['Gusto'] = 1 if respuesta in ['¡La amé!', 'Sí me gustó'] else 0
    
    if dfregistro.empty:
        dfregistro = pd.DataFrame([nuevo_registro])
    else:
        dfregistro = pd.concat([dfregistro, pd.DataFrame([nuevo_registro])], ignore_index=True)

    # Definir los nombres de las columnas a actualizar
    columnas = ['U_Accion', 'U_Aventura', 'U_Animacion', 'U_Comedia', 
            'U_Crimen', 'U_Documental', 'U_Drama', 'U_Familiar', 'U_Fantasia', 
            'U_Historia', 'U_Horror', 'U_Musical', 'U_Misterio', 'U_Romance', 
            'U_Ciencia_Ficcion', 'U_Suspense', 'U_Guerra', 'U_Western']

    # Actualizar los valores de las columnas correspondientes al usuario con el ID especificado
    dfusuarios.loc[dfusuarios['ID_Usuario'] == user_id, columnas] = arreglo_perfil

    dfusuarios.to_csv(r"usuarios.csv", index=False)
    
    with output_vista:
        clear_output()
        #print("Perfil y registro actualizados:")
        #print(arreglo_perfil)

In [10]:
def eliminar_pelicula(seleccion):
    global peliculasrecomendadas, recomendacioncartelera
    pelicula_seleccionada = peliculasrecomendadas.iloc[seleccion]
    
    recomendacioncartelera = recomendacioncartelera.drop(recomendacioncartelera[recomendacioncartelera['Titulo'] == pelicula_seleccionada['Titulo']].index)
    peliculasrecomendadas = peliculasrecomendadas.drop(peliculasrecomendadas[peliculasrecomendadas['Titulo'] == pelicula_seleccionada['Titulo']].index)
    peliculasrecomendadas.reset_index(drop=True, inplace=True)
    peliculasrecomendadas['ID'] = range(1, len(peliculasrecomendadas) + 1)

    #with output_vista:
        #print("Pelicula eliminada de listas recomendadas y cartelera.")

In [11]:
def mostrar_menu():
    output_vista = widgets.Output()

    def manejar_accion(btn):
        with output_vista:
            clear_output()
            if btn.description == 'Recomiéndame una película':
                ObtenerRecomendacion()
            elif btn.description == 'Calificar una película':
                CalificarPelicula()
            elif btn.description == 'Salir':
                print("Gracias por utilizar la aplicación, hasta pronto.")
                display(Javascript('window.location.href = "login.ipynb";'))
                return
    
    btn_obtener = widgets.Button(description="Recomiéndame una película", layout=widgets.Layout(width='300px'))
    btn_calificar = widgets.Button(description="Calificar una película", layout=widgets.Layout(width='300px'))
    btn_salir = widgets.Button(description="Salir", layout=widgets.Layout(width='300px'))
    
    btn_obtener.on_click(manejar_accion)
    btn_calificar.on_click(manejar_accion)
    btn_salir.on_click(manejar_accion)

    display(widgets.Label(f"Selecciona la opción que quieras realizar:"), btn_obtener, btn_calificar, btn_salir, output_vista)

    #with output_vista:
        #print(arreglo_perfil)

In [12]:
# Cargar el ID_Usuario desde el archivo JSON
with open('user_id.json', 'r') as f:
    user_id = json.load(f)['ID_Usuario']

# Cargar el modelo desde el archivo
with open('recomendadorknn.pkl', 'rb') as file:
    recomendadorknn = pickle.load(file)

# Leer el archivo CSV
dfusuarios = pd.read_csv('usuarios.csv')

# Obtener la información del usuario
user = dfusuarios[dfusuarios['ID_Usuario'] == user_id].iloc[0]

nombre = user['Nombre']
anio_nac = user['Anio_nac']
mes_nac = user['Mes_nac']
dia_nac = user['Dia_nac']

# Calcular la edad del usuario
fecha_nacimiento = datetime.date(anio_nac, mes_nac, dia_nac)
hoy = datetime.date.today()
edad_usuario = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

# Mostrar mensaje de bienvenida y la edad del usuario
print(f"Bienvenido {nombre}")
#print(f"Tienes {edad_usuario} años.")

Bienvenido Cesar


In [13]:
# Lista de géneros de películas
generos = ['Acción', 'Aventura', 'Animación', 'Comedia', 'Crimen', 'Documental',
           'Drama', 'Familiar', 'Fantasía', 'Historia', 'Horror', 'Musical',
           'Misterio', 'Romance', 'Ciencia Ficción', 'Suspense', 'Guerra', 'Western']

In [14]:
columnas1 = [
    "ID", "Titulo", "Clasificacion", "O_Titulo", "Duracion",
    "p_accion", "p_aventura", "p_animacion", "p_comedia", "p_crimen",
    "p_documental", "p_drama", "p_familia", "p_fantasia", "p_historia",
    "p_terror", "p_musica", "p_misterio", "p_romance", "p_ciencia_ficcion",
    "p_suspense", "p_belica", "p_western", "Calificacion"
]

# Crear un DataFrame vacío con las columnas definidas
peliculasrecomendadas = pd.DataFrame(columns=columnas1)

columnas2 = [
    "u_accion", "u_aventura", "u_animacion", "u_comedia", "u_crimen",
    "u_documental", "u_drama", "u_familia", "u_fantasia", "u_historia",
    "u_terror", "u_musica", "u_misterio", "u_romance", "u_ciencia_ficcion",
    "u_suspense", "u_belica", "u_western",
    "p_accion", "p_aventura", "p_animacion", "p_comedia", "p_crimen",
    "p_documental", "p_drama", "p_familia", "p_fantasia", "p_historia",
    "p_terror", "p_musica", "p_misterio", "p_romance", "p_ciencia_ficcion",
    "p_suspense", "p_belica", "p_western", "Calificacion", "Gusto"
]

# Crear un DataFrame vacío con las columnas definidas
#dfregistro = pd.DataFrame(columns=columnas2)

columnas_dfregistro = [f'u_{g.replace(" ", "_").lower()}' for g in generos] + \
                      [f'p_{g.replace(" ", "_").lower()}' for g in generos] + \
                      ['Calificacion', 'Gusto']
dfregistro = pd.DataFrame(columns=columnas_dfregistro) if 'dfregistro' not in locals() else dfregistro


In [15]:
cartelera = pd.read_csv(r"peliculas_clasificadas.csv")
#cartelera.head(15)

cartelera.rename(columns={cartelera.columns[0]: 'Titulo'}, inplace=True)
cartelera.rename(columns={cartelera.columns[1]: 'Clasificacion'}, inplace=True)
cartelera.rename(columns={cartelera.columns[2]: 'O_Titulo'}, inplace=True)
cartelera.rename(columns={cartelera.columns[3]: 'Duracion'}, inplace=True)
cartelera.rename(columns={cartelera.columns[-11]: 'Calificacion'}, inplace=True)
cartelera[cartelera.columns[-11]] = (cartelera[cartelera.columns[-11]] * 0.01).round(2)

# Calculamos el promedio excluyendo las calificaciones que son cero y redondeamos a 2 decimales
promedio_no_cero = cartelera[cartelera['Calificacion'] > 0]['Calificacion'].mean().round(2)

# Reemplazamos los valores cero en la columna 'Calificacion' con el promedio calculado
cartelera['Calificacion'] = cartelera['Calificacion'].replace(0, promedio_no_cero)

# Muestra las primeras filas para verificar los cambios
#cartelera.head(10)

cartelera = filtrar_peliculas_por_edad(cartelera, edad_usuario)
#cartelera.head(5)

In [16]:
usuario = dfusuarios[dfusuarios['ID_Usuario'] == user_id]

# Extraer los valores de los atributos de interés y convertirlos en una lista
arreglo_perfil = usuario.loc[:, 'U_Accion':'U_Western'].values.flatten().tolist()
arreglo_perfil = np.round(arreglo_perfil, 2)

# Obtener las películas recomendadas
recomendacioncartelera = predict_movies2(arreglo_perfil, cartelera)

In [17]:
output_vista = widgets.Output()

display(output_vista)

mostrar_menu()

Output()

Label(value='Selecciona la opción que quieras realizar:')

Button(description='Recomiéndame una película', layout=Layout(width='300px'), style=ButtonStyle())

Button(description='Calificar una película', layout=Layout(width='300px'), style=ButtonStyle())

Button(description='Salir', layout=Layout(width='300px'), style=ButtonStyle())

Output()

In [18]:
#peliculasrecomendadas.head()

In [19]:
#dfregistro.head()

,u_acción,u_aventura,u_animación,u_comedia,u_crimen,u_documental,u_drama,u_familiar,u_fantasía,u_historia,u_horror,u_musical,u_misterio,u_romance,u_ciencia_ficción,u_suspense,u_guerra,u_western,p_acción,p_aventura,p_animación,p_comedia,p_crimen,p_documental,p_drama,p_familiar,p_fantasía,p_historia,p_horror,p_musical,p_misterio,p_romance,p_ciencia_ficción,p_suspense,p_guerra,p_western,Calificacion,Gusto


In [20]:
#arreglo_perfil

array([0.95, 0.95, 0.5 , 0.95, 0.5 , 0.4 , 0.  , 0.  , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.  , 0.7 , 0.  , 0.5 , 0.5 ])

In [21]:
#dfusuarios.head()

,ID_Usuario,Nombre,Password,Sexo,Anio_nac,Mes_nac,Dia_nac,U_Accion,U_Aventura,U_Animacion,U_Comedia,U_Crimen,U_Documental,U_Drama,U_Familiar,U_Fantasia,U_Historia,U_Horror,U_Musical,U_Misterio,U_Romance,U_Ciencia_Ficcion,U_Suspense,U_Guerra,U_Western
0,100001,Ariel,ariel,1,1991,3,3,0.15,0.15,0.20,0.25,0.2,0.2,0.25,0.20,0.2,0.20,0.2,0.2,0.2,0.90,0.25,0.20,0.20,0.2
1,100002,Marco,marco,1,1996,4,1,0.50,0.60,0.50,0.50,0.5,0.6,0.90,0.55,0.5,0.70,0.5,0.5,0.5,0.50,0.50,0.55,0.50,0.5
2,100003,Ruben,ruben,1,2010,5,10,1.00,0.10,0.95,0.05,0.9,0.0,0.80,0.00,0.8,0.55,0.5,0.5,0.5,0.50,0.55,0.55,0.55,0.5
3,100004,A,a,1,2022,9,14,0.85,1.00,1.00,0.55,0.0,0.0,0.00,0.65,1.0,0.00,0.0,0.5,0.5,0.55,0.90,0.50,0.50,0.5
4,100005,Cesar,cesar,1,1996,8,15,0.95,0.95,0.50,0.95,0.5,0.4,0.00,0.00,0.5,0.50,0.5,0.5,0.5,0.00,0.70,0.00,0.50,0.5
